In [ ]:
import math
import cmath
import functools
import operator
import numpy as np
import sympy
import mpmath
import scipy
import qutip 
import vpython

###

def combos(a,b):
    f = math.factorial
    return f(a) / f(b) / f(a-b)

###

class Variable:    
    def __init__(self):
        self.value = None
        self.dependencies = []
        self.touched = 0

    def tie(self, other, transformation):
        self.dependencies.append({"other": other, "transformation": transformation})

    def plug(self, value):
        self.value = value
        if self.touched == 0:
            self.touched = 1
        elif self.touched == 1:
            self.touched = 0
        for dependency in self.dependencies:
            other, transformation = dependency["other"], dependency["transformation"]
            if other.touched != self.touched:
                other.plug(transformation(self.value))

    def __str__(self):
        return str(self.value)

###

class Sphere:
    def __init__(self, x, y, z):
        self.center = [x, y, z]
        self.vsphere = None
        self.vstars = None
        
        self.state = Variable()
        self.vector = Variable()
        self.polynomial = Variable()
        self.roots = Variable()
        self.xyzs = Variable()
        
        def state_to_vector(state):
            return state.full().T[0]
        self.state.tie(self.vector, state_to_vector)

        def vector_to_state(vector):
            return qutip.Qobj(vector)
        self.vector.tie(self.state, vector_to_state)

        def vector_to_polynomial(vector):
            polynomial = vector.tolist()
            return [(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) * polynomial[i] for i in range(len(polynomial))] 
        self.vector.tie(self.polynomial, vector_to_polynomial)

        def polynomial_to_vector(polynomial):
            coordinates = [polynomial[i]/(((-1)**i) * math.sqrt(combos(len(polynomial)-1,i))) for i in range(len(polynomial))]
            return np.array(coordinates)
        self.polynomial.tie(self.vector, polynomial_to_vector)

        def polynomial_to_roots(polynomial):
            try:
                return [np.conjugate(complex(root)) for root in mpmath.polyroots(polynomial)]
            except:
                return [complex(0,0) for i in range(len(polynomial)-1)]
        self.polynomial.tie(self.roots, polynomial_to_roots)

        def roots_to_polynomial(roots):
            s = sympy.symbols("s")
            polynomial = sympy.Poly(functools.reduce(lambda a, b: a*b, [s-np.conjugate(root) for root in roots]), domain="CC")
            return [complex(c) for c in polynomial.coeffs()]
        self.roots.tie(self.polynomial, roots_to_polynomial)

        def roots_to_xyzs(roots):
            def root_to_xyz(root):
                if root == float('inf'):
                    return [0,0,1]
                x = root.real
                y = root.imag
                return [(2*x)/(1.+(x**2)+(y**2)),\
                        (2*y)/(1.+(x**2)+(y**2)),\
                        (-1.+(x**2)+(y**2))/(1.+(x**2)+(y**2))]
            return [root_to_xyz(root) for root in roots]
        self.roots.tie(self.xyzs, roots_to_xyzs)

        def xyzs_to_roots(xyzs):
            def xyz_to_root(xyz):
                x, y, z = xyz[0], xyz[1], xyz[2]
                if z == 1:
                    return float('inf') 
                else:
                    return complex(x/(1-z), y/(1-z))
            return [xyz_to_root(xyz) for xyz in xyzs]
        self.xyzs.tie(self.roots, xyzs_to_roots)
        
    def visualize(self):
        if self.state.value != None:
            if self.vsphere == None:
                self.vsphere = vpython.sphere()
            if self.vstars == None:
                self.vstars = [vpython.sphere() for i in range(len(self.xyzs.value))]
            vpython.rate(100)
            self.vsphere.pos = vpython.vector(*self.center)
            self.vsphere.radius = 1
            self.vsphere.color = vpython.color.blue
            self.vsphere.opacity = 0.4
            for i in range(len(self.vstars)):
                self.vstars[i].pos = vpython.vector(*self.xyzs.value[i])
                self.vstars[i].radius = 0.1
                self.vstars[i].color = vpython.color.white
                self.vstars[i].opacity = 0.8
    
    def spin_operators(self):
        if self.state.value != None:
            n = len(self.vector.value)
            spin = (n-1.)/2.
            return {"X": qutip.jmat(spin, "x"),\
                    "Y": qutip.jmat(spin, "y"),\
                    "Z": qutip.jmat(spin, "z"),\
                    "+": qutip.jmat(spin, "+"),\
                    "-": qutip.jmat(spin, "-")}
    
    def evolve(self, hermitian, inverse=False, dt=0.007):
        unitary = qutip.Qobj(scipy.linalg.expm(-2*math.pi*complex(0,1)*hermitian.full()*dt))
        if inverse:
            unitary = unitary.dag()
        self.state.plug(unitary*self.state.value)

vpython.scene.width = 900
vpython.scene.height = 900

sphere = Sphere(0,0,0)

def keyboard(event):
    global sphere
    key = event.key
    spin_ops = sphere.spin_operators()
    if key == "a":   #-x for A
        sphere.evolve(spin_ops["X"], inverse=True)
    elif key == "d": #+x for A
        sphere.evolve(spin_ops["X"], inverse=False)
    elif key == "s": #-z for A
        sphere.evolve(spin_ops["Z"], inverse=True)
    elif key == "w": #+z for A
        sphere.evolve(spin_ops["Z"], inverse=False)
    elif key == "z": #-y for A
        sphere.evolve(spin_ops["Y"], inverse=True)
    elif key == "x": #+y for A
        sphere.evolve(spin_ops["Y"], inverse=False)
vpython.scene.bind('keydown', keyboard)

sphere.state.plug(qutip.rand_ket(6))

while True:
    sphere.visualize()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>